<a href="https://colab.research.google.com/github/SpyroRock/U-net-pixel-reconstruction/blob/master/U_net_pixel_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SpyroRock/U-net-pixel-reconstruction

Cloning into 'U-net-pixel-reconstruction'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2038 (delta 1), reused 1 (delta 0), pack-reused 2033
Receiving objects: 100% (2038/2038), 250.52 MiB | 33.79 MiB/s, done.
Resolving deltas: 100% (1142/1142), done.
Checking out files: 100% (16/16), done.


In [2]:
%cd U-net-pixel-reconstruction

/content/U-net-pixel-reconstruction


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 51.1MB 61kB/s 
     |████████████████████████████████| 307kB 54.8MB/s 
     |████████████████████████████████| 573kB 56.3MB/s 
     |████████████████████████████████| 3.3MB 54.1MB/s 
     |████████████████████████████████| 51kB 9.8MB/s 
     |████████████████████████████████| 3.3MB 50.3MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp36-none-any.whl size=29072 sha256=fd0d312940220176be5852665debb54d8cf20b2596a9738d555572803b9bad50
  Stored in directory: /root/.cache/pip/wheels/d0/bb/c0/6aad88b38f6e46db048bed4cccb904a5897055a8ab6fbd4dfc
Successfully built keras-segmentation
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: google-auth 1.25.0 has requirement setuptools>=40.3.0, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: 

In [1]:
from unet_model_construction import get_unet
import keras
from keras.models import Model
from keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
from numpy import load, save
from sklearn.model_selection import train_test_split
import pickle

ModuleNotFoundError: ignored

In [ ]:
img_height = 256
img_width = 256

img_height_test = 256
img_width_test = 256

speckle_data = load('speckle_cluster_case2.npy')
print(speckle_data.shape)
speckle_labels = load('symbol_cluster_case2.npy')
print(speckle_labels.shape)